In [ ]:
import os

# Define the language as a single variable
language = 'kor'

# Use the language variable to construct the folder path
folder_path = f'new_translationss/{language}'

# List all files in the directory
for root, dirs, files in os.walk(folder_path):
    for file in files:
        print(os.path.join(root, file))


In [ ]:
import os

# Use the language variable to construct the folder path
folder_path = f'new_translationss/{language}'

# Initialize a dictionary to store the line counts for each file
file_line_counts = {}

# Loop through all files in the specified directory
for root, dirs, files in os.walk(folder_path):
    for file in files:
        # Skip .DS_Store and any other hidden files
        if file == '.DS_Store' or file.startswith('.'):
            continue

        file_path = os.path.join(root, file)
        
        # Count lines in the file, ignoring decode errors
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                line_count = sum(1 for line in f)
            # Store the line count in the dictionary
            file_line_counts[file] = line_count
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")

# Check if all files have the same line count
line_counts = list(file_line_counts.values())
all_equal = all(count == line_counts[0] for count in line_counts)

# Print results
if all_equal:
    print("All files have the same number of lines:", line_counts[0])
else:
    print("Mismatch in line counts detected:")
    for file, count in file_line_counts.items():
        print(f"{file}: {count} lines")


In [ ]:
###KOREAN
import sacrebleu
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-korean')

def tokenize_with_transformers(text):
    """Tokenize text using Hugging Face's BERT tokenizer."""
    return tokenizer.tokenize(text)

def detokenize_with_transformers(tokens):
    """Detokenize using Hugging Face's BERT tokenizer."""
    return tokenizer.convert_tokens_to_string(tokens)

def compute_bleu(reference_file, hypothesis_file):
    with open(reference_file, 'r', encoding='utf-8') as ref_file:
        references = [ref_file.read().strip().split('\n')]
    with open(hypothesis_file, 'r', encoding='utf-8') as hyp_file:
        hypotheses = hyp_file.read().strip().split('\n')

    # Tokenize the references and hypotheses using the transformers tokenizer
    tokenized_references = [tokenize_with_transformers(ref) for ref in references[0]]
    tokenized_hypotheses = [tokenize_with_transformers(hyp) for hyp in hypotheses]

    # Detokenize back to strings for BLEU computation
    detokenized_references = [detokenize_with_transformers(ref) for ref in tokenized_references]
    detokenized_hypotheses = [detokenize_with_transformers(hyp) for hyp in tokenized_hypotheses]

    # Compute BLEU score
    bleu = sacrebleu.corpus_bleu(detokenized_hypotheses, [detokenized_references])
    return bleu.score

def compute_chrf(reference_file, hypothesis_file):
    with open(reference_file, 'r', encoding='utf-8') as ref_file:
        references = [ref_file.read().strip().split('\n')]
    with open(hypothesis_file, 'r', encoding='utf-8') as hyp_file:
        hypotheses = hyp_file.read().strip().split('\n')

    # Tokenize and detokenize for chrF
    tokenized_references = [tokenize_with_transformers(ref) for ref in references[0]]
    tokenized_hypotheses = [tokenize_with_transformers(hyp) for hyp in hypotheses]
    detokenized_references = [detokenize_with_transformers(ref) for ref in tokenized_references]
    detokenized_hypotheses = [detokenize_with_transformers(hyp) for hyp in tokenized_hypotheses]

    # Compute chrF score
    chrf = sacrebleu.corpus_chrf(detokenized_hypotheses, [detokenized_references])
    return chrf.score

def compute_ter(reference_file, hypothesis_file):
    with open(reference_file, 'r', encoding='utf-8') as ref_file:
        references = [ref_file.read().strip().split('\n')]
    with open(hypothesis_file, 'r', encoding='utf-8') as hyp_file:
        hypotheses = hyp_file.read().strip().split('\n')

    # Tokenize and detokenize for TER
    tokenized_references = [tokenize_with_transformers(ref) for ref in references[0]]
    tokenized_hypotheses = [tokenize_with_transformers(hyp) for hyp in hypotheses]
    detokenized_references = [detokenize_with_transformers(ref) for ref in tokenized_references]
    detokenized_hypotheses = [detokenize_with_transformers(hyp) for hyp in tokenized_hypotheses]

    # Compute TER score
    ter = sacrebleu.corpus_ter(detokenized_hypotheses, [detokenized_references])
    return ter.score

plots_folder = os.path.join(folder_path, "plots")
os.makedirs(plots_folder, exist_ok=True)

def evaluate_all_metrics(reference_file, folder_path, thresholds={'BLEU': 20, 'chrF': 50, 'TER': 60}):
    models = []
    bleu_scores = []
    chrf_scores = []
    ter_scores = []
    bleu_pass_counts = []
    chrf_pass_counts = []
    ter_pass_counts = []

    excluded_files = {'eng.txt', f'{language}.txt'}
    hypothesis_files = [f for f in os.listdir(folder_path) if f.endswith('.txt') and f not in excluded_files]

    for hypothesis_file in hypothesis_files:
        hyp_path = os.path.join(folder_path, hypothesis_file)
        models.append(hypothesis_file)
        
        bleu_score = compute_bleu(reference_file, hyp_path)
        chrf_score = compute_chrf(reference_file, hyp_path)
        ter_score = compute_ter(reference_file, hyp_path)
        
        bleu_scores.append(bleu_score)
        chrf_scores.append(chrf_score)
        ter_scores.append(ter_score)

        # Count how many scores exceed thresholds
        bleu_pass_counts.append(bleu_score >= thresholds['BLEU'])
        chrf_pass_counts.append(chrf_score >= thresholds['chrF'])
        ter_pass_counts.append(ter_score <= thresholds['TER'])  # For TER, lower is better

        print(f"{hypothesis_file}:")
        print(f"  BLEU score: {bleu_score:.2f}")
        print(f"  chrF score: {chrf_score:.2f}")
        print(f"  TER score: {ter_score:.2f}")
    
    # Convert results into a DataFrame for better analysis
    results_df = pd.DataFrame({
        'Model': models,
        'BLEU Score': bleu_scores,
        'chrF Score': chrf_scores,
        'TER Score': ter_scores,
        'BLEU Above Threshold': bleu_pass_counts,
        'chrF Above Threshold': chrf_pass_counts,
        'TER Below Threshold': ter_pass_counts
    })

    # Calculate the number and percentage of models meeting each threshold
    total_files = len(models)
    bleu_pass_count = sum(bleu_pass_counts)
    chrf_pass_count = sum(chrf_pass_counts)
    ter_pass_count = sum(ter_pass_counts)

    summary = {
        'Metric': ['BLEU', 'chrF', 'TER'],
        'Pass Count': [bleu_pass_count, chrf_pass_count, ter_pass_count],
        'Percentage': [
            bleu_pass_count / total_files * 100,
            chrf_pass_count / total_files * 100,
            ter_pass_count / total_files * 100
        ]
    }
    summary_df = pd.DataFrame(summary)
    
    # Display DataFrames
    print("\nDetailed Model Performance:")
    print(results_df)
    print("\nSummary of Threshold Passing Rates:")
    print(summary_df)
    
    # Visualize scores
    visualize_metrics(models, bleu_scores, chrf_scores, ter_scores)

def visualize_metrics(models, bleu_scores, chrf_scores, ter_scores):
    x = np.arange(len(models))
    width = 0.3  # Bar width

    # Plot BLEU scores
    fig, ax1 = plt.subplots(figsize=(10, 6))
    ax1.bar(x, bleu_scores, width, color='blue')
    ax1.set_xlabel('Models')
    ax1.set_title('BLEU Scores by Model')
    ax1.set_xticks(x)
    ax1.set_xticklabels(models, rotation=45, ha='right')
    ax1.set_ylabel('BLEU Score')
    plt.tight_layout()
    bleu_plot_path = os.path.join(plots_folder, f"bleu_scores_{language}.png")
    plt.savefig(bleu_plot_path)
    plt.show()

    # Plot chrF scores
    fig, ax2 = plt.subplots(figsize=(10, 6))
    ax2.bar(x, chrf_scores, width, color='green')
    ax2.set_xlabel('Models')
    ax2.set_title('chrF Scores by Model')
    ax2.set_xticks(x)
    ax2.set_xticklabels(models, rotation=45, ha='right')
    ax2.set_ylabel('chrF Score')
    plt.tight_layout()
    chrf_plot_path = os.path.join(plots_folder, f"chrf_scores_{language}.png")
    plt.savefig(chrf_plot_path)
    plt.show()

    # Plot TER scores
    fig, ax3 = plt.subplots(figsize=(10, 6))
    ax3.bar(x, ter_scores, width, color='red')
    ax3.set_xlabel('Models')
    ax3.set_title('TER Scores by Model')
    ax3.set_xticks(x)
    ax3.set_xticklabels(models, rotation=45, ha='right')
    ax3.set_ylabel('TER Score')
    plt.tight_layout()
    ter_plot_path = os.path.join(plots_folder, f"ter_scores_{language}.png")
    plt.savefig(ter_plot_path)
    plt.show()

# Paths
reference_path = f'new_translationss/{language}/{language}.txt'
folder_path = f'new_translationss/{language}'

# Run the comprehensive evaluation and visualization
evaluate_all_metrics(reference_path, folder_path)
### CHINISE